In [ ]:
!pip install ortools
!pip install dtale

In [1]:
import pandas as pd
import numpy as np
#from ortools.sat.python import cp_model
#from dtale import show
#import dtale.app as dtale_app
#dtale_app.USE_COLAB = True

In [2]:
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (38/38), done.
/content/cloned-repo
 CP.ipynb  'item master.xlsx'   orders.xlsx   README.md


In [3]:
df=pd.read_excel('orders.xlsx')

In [4]:
df = df[df['Line']=='Tmmthpkhti Limi']
del df['Line']
df['due'] = df['Dispatch Date'] - df['So Date'].min()
df['due'] = df['due'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']

In [ ]:
show(df,ignore_duplicate=True)

https://pi6pg94wws-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/3

In [14]:
df.pop?

In [6]:
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [5]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]
sodate = df['SO Date'] - df['SO Date'].min()
sodate = sodate.dt.days
deldate = df['due']

In [12]:
df.to_numpy()

array([[17023287, 'mWI', Timestamp('2019-01-14 00:00:00'), ..., 1024, 0,
        '17023287gd29904300iX'],
       [17023287, 'mWI', Timestamp('2019-01-14 00:00:00'), ..., 1024, 0,
        '17023287gd29907800iX'],
       [17023287, 'mWI', Timestamp('2019-01-14 00:00:00'), ..., 1024, 0,
        '17023287gd39000800iX'],
       ...,
       [17023833, 'mWI', Timestamp('2019-04-23 00:00:00'), ..., 1024, 84,
        '17023833gd55102400iX'],
       [17023833, 'mWI', Timestamp('2019-04-23 00:00:00'), ..., 1024, 84,
        '17023833gd30212200iX'],
       ['RdPJumw4', 'dmdihtic', Timestamp('2019-04-15 00:00:00'), ...,
        1025, 90, 'RdPJumw4gd05010k']], dtype=object)

In [7]:
pa

,FG Code,Product,Line,Speed,Cs size,Output,Manpower/hr
0,gd28001700iX,grishdimt glumridi TPST 17g (0.6) mZ,1,120.0,720,6,1.5
1,gd28002400iX,grishdimt glumridi TPST 24g(0.85) mZ,1,120.0,720,6,1.5
2,gd24312000iX,Prmdisi cavity prmtictimm Tpst 125+20g,1,60.0,72,30,2.75
3,gd246100W0iX,PRmdISi TmmTHPASTi (WHITmG) 100 G -,1,60.0,72,30,3.125
4,gd247100d0iX,PRmdISi TmmTHPASTi (d.SmdA ) 100G -,1,60.0,72,30,3.125
...,...,...,...,...,...,...,...
1698,gd29900401iX,daxidud Sicurity GiL TPST 4.25 G-0.15mZ,6,1.0,1000,0.036,1.5
1699,gd30100800iX,maturidimt Amticavity gLd. TPST 0.28 mZ,6,1.0,1000,0.036,0.5
1700,gd39000800iX,grishdimt Amticavity gLd GiL TPST 0.28mZ,6,1.0,1000,0.036,0.5
1701,gd43200800iX,grishdimt Pridiud AC TPST 8G-0.28mZ-AdA,6,1.0,1000,0.036,0.625


In [ ]:
df1